# Settima lezione. Parte A

Importeremo i file da una repositoria su github. Spezziamo la URL in varie parti per chiarezza.

In [1]:
baseURL    = 'https://raw.githubusercontent.com/'
user       = 'domenicozambella/'
repository = 'BioTeIndu19/'
branch     = 'master/'

Importiamo gli stessi dati della lezione scorsa.

In [2]:
import pandas as pd
file       = 'dati/mm1.csv'
df = pd.read_csv( baseURL + user + repository + branch + file )

Importiamo la libreria grafica e settiamo alcuni parametri

In [3]:
from bokeh.plotting import figure, show, output_notebook
param = dict(width = 700, height = 350,
             tools = 'wheel_zoom, reset,pan, box_zoom',
            )
output_notebook()

Loading BokehJS ...

Facciamo un preliminare plot dei dati

In [4]:
p = figure(x_axis_label='Concentrazione substrato',
           y_axis_label='Velocità di reazione',
           **param,
           )
p.circle( 's', 'v', source=df )
show( p )

<hr><br>

# Regressione non-lineare

Ricordo l'equazione di Michaelis-Menten

$$
v = \frac{V_\text{max}\cdot s}{K_\text{m} + s}
$$


Possiamo calcolare $V_\text{max}$ e $K_\text{m}$ senza prima linearizzare. Useremo una regressione non-lineare.

Ci sono moltissime possibilità. Useremo la funzione `curve_fit()` della libreria `scipy.optimize`. 

Quindi importiamo `curve_fit()` dalla libreria e definiamo la funzione (che chiamiamo `mm`) che calcola i valori teorici.

In [5]:
from scipy.optimize import curve_fit
mm = lambda s, Vmax, Km:   Vmax * s / ( Km + s)

La variabile `mm` sarà uno degli argomenti da fornire a `curve_fit()`. La prima variabile di `mm` si considera contenere i valori indipendenti. Tutte le altre variabili verranno considerati come parametri.  La funzione `curve_fit()` ricerca i valori dei parametri che minimizzano la somma dei quadrati delle distanze tra i valori teorici e quelli misurati.

Il minimo viene ricercato con prove successive. Il punto di partenza è arbitrario. È possibile inserire un punto di partenza. Non è necessario, ma nel caso esistano più minimi locali conviene scegliere un valore ragionevolmente vicino al risultato previsto. (In teoria la procedura potrebbe trovare un minimo locale diverso.)

La funzione `curve_fit()` restituisce due valori. Il primo è la tupla di parametri. Il secondo verrà preso in considerazione qui sotto per il calcolo dell'errore. Per il momento lo ignoriamo.

In [6]:
(Vmax, Km), _ = curve_fit( mm, df['s'], df['v'] )

Plottiamo il risultato aggiungendo il grafico della curva alla figura `p` introdotta sopra.

In [7]:
from numpy import linspace       
xmax = df['s'].max()             # Estremo destro dei valori della x del grafico
s = linspace( 0, xmax, 100  )    # Un array di 100 punti tra xmin e xmax
v = mm(s, Vmax, Km)              # I valori della curva interpolante
p.title.text = 'Least square interpolation'
p.line(s, v)
show( p )

## I residui


Il modello con cui interpretiamo le i dati misurati $(s_i, v_i)$, per $i =1,\dots,n$, è il seguente:

$$
v_i = \frac{V_\text{max}\cdot s_i}{K_\text{m} + s_i} \ +\ \varepsilon_i
$$

Dove $\varepsilon_i$, è l'errore di misura (*rumore*) che assumiamo essere ottenuto da variabili aleatorie indipendenti distribuite normalmente.

Non conosciamo i veri valori di $V_\text{max}$ e $K_\text{m}$ ma la regressione ci fornisce delle stime  $\hat V_\text{max}$ e $\hat K_\text{m}$

I **residui** di un'interpolazione sono le differenze $\hat\varepsilon_i$ tra i valori misurati e quelli previsti dalla curva interpolante. Questi sono una stima del rumore $\varepsilon_i$

$$
\hat\varepsilon_i =  v_i\ -\ \frac{\hat V_\text{max}\cdot s_i}{\hat K_\text{m} + s_i}
$$

Aggiungiamo al dataframe i valori dei residui 

In [8]:
df['res'] = df['v'] - Vmax * df['s'] / ( Km + df['s'] )

Facciamo un istrogramma per valutare graficamente come sono distribuiti i residui.

In [9]:
from numpy import histogram
freq, edges = histogram(df['res'], bins=5)

p2 = figure( **param )
p2.quad(top=freq, bottom=0, left=edges[:-1], right=edges[1:],
           fill_color="navy", line_color="white", alpha=0.5)
show( p2 )

Facciamo un test di normalità. 

Tra i molti possibili test scegliamo il test di Shapiro–Wilk (giusto un esempio).

L'ipotesi nulla di questo test ̀e che i dati provengano da una popolazione distribuita normalmente.

La funzione `shapiro` della libreria `scipy.stats` restituisce una coppia: valore della statistica, p-valore.

In [10]:
from scipy.stats import shapiro
shapiro( df['res'] )

(0.9237818121910095, 0.2819909155368805)

Il p-valore è abbastanza alto quindi NON rifiutiamo l'ipotesi nulla (in questo casi, una buona notizia) .

Non è improbabille che i dati provengano da una popolazione normale.

## Intervalli di confidenza

In [11]:
(Vmax, Km), cov = curve_fit( mm, df['s'], df['v'] )
(varVmax, _), (_, varKm) = cov

In [12]:
from scipy.stats import t
alpha  = 0.05                  # 95% confidence interval
n      = len(df)               # number of data points
par    = 2                     # number of parameters
dof    = n - par               # number of degrees of freedom
talpha = t.ppf(1-alpha/2, dof) # student-t value for the confidence level

In [13]:
errVmax = talpha * varVmax**0.5
errKm   = talpha * varKm**0.5

Giusto per esercizio, aggiungiamo al grafico precedente una rappresentazione dell'intervallo di confidenza. (Purtroppo al momento bokeh non ha una funzione naturale per ombreggiare una regione delimitata due curve.)

In [14]:
sup = mm(s, Vmax - errVmax, Km + errKm) # le y della curva superiore
inf = mm(s, Vmax + errVmax, Km - errKm) # le y della curva inferiore

# Per ombreggiare l'areatra due curve definiamo un poligono che coloreremo.
# Le siste x e y sono le coordinate dei vertici di questi poligoni.
x = list(s)   + list(s[::-1])   # s concatenato con s in ordine inverso
y = list(inf) + list(sup[::-1]) # inf concatenato con sup in ordine inverso
p.patch(x, y, alpha= 0.2 )

p.title.text = 'Least square interpolation with 95% confidence interval'
p.text(60, 1, text=['Vmax = {:.1f} ± {:.1f}\n    Km = {:.1f} ± {:.1f}'.format(Vmax, errVmax, Km, errKm)] )
show( p )

Fine

<hr><hr><hr>

La seguente cella importa file di stile HTML (può anche essere ignorata)

In [15]:
import requests
file = 'lezioni/style/custom.css'
from IPython.core.display import HTML
html_style = requests.get( baseURL + user + repository + branch + file ).text
HTML( html_style )